<div style="border-radius: 10px; box-shadow: 0px 0px 2px; border: 1px solid; background:#eaeaea; position: relative; padding: 10px; padding-left: 10px;">
<h7 style="color:black; margin-bottom:20px"> 

# <span style="color:#143a51">Определение уровня потребительской лояльности<span> 
    
---
---   
    
**Цель:**
необходимо для большой телекоммуникационной компании интерпретировать результаты NPS-опросов, выявить наиболее лояльных пользователей.  
--- 
[Ссылка на дашборд](https://public.tableau.com/app/profile/kristina2465/viz/__16760144028600/Dashboard5)
    
[Ссылка на презентацию](https://disk.yandex.ru/i/JHlFL26Py6GPXA)

**`Витрина данных:`**

`user_id` - идентификатор клиента    
`lt_day` - количество дней «жизни» клиента    
`is_new` - поле хранит информацию о том, является ли клиент новым    
`age` -	возраст    
`gender_segment` - пол (для удобства работы с полем преобразуйте значения в текстовый вид) `os_name` -	тип операционной системы  
`cpe_type_name` - тип устройства     
`country` -	страна проживания     
`city` - город проживания    
`age_segment` -	возрастной сегмент    
`traffic_segment`  - сегмент по объёму потребляемого трафика   
`lifetime_segment` - сегмент по количеству дней «жизни»   
`nps_score` - оценка клиента в NPS-опросе    
`nps_group` - поле хранит информацию о том, к какой группе относится оценка клиента в опросе   


In [1]:
# импортируем библиотеки
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
# параметры подключения
path_to_db = '/datasets/telecomm_csi.db'
engine = create_engine(f'sqlite:///{path_to_db}', echo = False)

In [3]:
# запрос для витрины данных, в дальнейшем используемой для создания дашборда
query = """
SELECT user_id,
       lt_day,
       CASE
           WHEN lt_day <= 365 THEN 'новый пользователь'
           WHEN lt_day > 365 THEN 'старый пользователь'
       END AS is_new,
       age,
       CASE 
           WHEN gender_segment = 1 THEN 'жен'
           WHEN gender_segment = 0 THEN 'муж'
       END AS gender_segment,
       os_name,
       cpe_type_name,
       l.country,
       l.city,
       a.title AS age_segment,
       t.title AS traffic_segment,
       li.title AS lifetime_segment,
       nps_score,
       CASE
           WHEN nps_score >= 9 THEN 'сторонники'
           WHEN nps_score <= 6 THEN 'критики'
           WHEN 6 < nps_score < 9 THEN 'нейтралы'
       END AS nps_group
       
FROM user AS u
LEFT JOIN location AS l ON u.location_id = l.location_id
LEFT JOIN age_segment AS a ON u.age_gr_id = a.age_gr_id
LEFT JOIN traffic_segment AS t ON u.tr_gr_id = t.tr_gr_id
LEFT JOIN lifetime_segment AS li ON u.lt_gr_id = li.lt_gr_id;
"""

In [4]:
# сохраняем в датафрейм
df = pd.read_sql(query, engine)
df.head()

,user_id,lt_day,is_new,age,gender_segment,os_name,cpe_type_name,country,city,age_segment,traffic_segment,lifetime_segment,nps_score,nps_group
0,A001A2,2320,старый пользователь,45.0,жен,ANDROID,SMARTPHONE,Россия,Уфа,05 45-54,04 1-5,08 36+,10,сторонники
1,A001WF,2344,старый пользователь,53.0,муж,ANDROID,SMARTPHONE,Россия,Киров,05 45-54,04 1-5,08 36+,10,сторонники
2,A003Q7,467,старый пользователь,57.0,муж,ANDROID,SMARTPHONE,Россия,Москва,06 55-64,08 20-25,06 13-24,10,сторонники
3,A004TB,4190,старый пользователь,44.0,жен,IOS,SMARTPHONE,Россия,РостовнаДону,04 35-44,03 0.1-1,08 36+,10,сторонники
4,A004XT,1163,старый пользователь,24.0,муж,ANDROID,SMARTPHONE,Россия,Рязань,02 16-24,05 5-10,08 36+,10,сторонники


In [5]:
# сохраняем датафрейм как CSV-файл
df.to_csv('telecomm_csi_tableau.csv', index=False)

**Дашборд и графики для презентации выполняем в Tableau Public**